# Ce fichier a pour but de préparer les datasets de test et d'entrainement

In [1]:
import numpy as np
import pandas as pd
import csv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy import SpotifyException

## Retirer colonnes inutiles et doublons

In [2]:
csv_filename = 'data/spotify_dataset.csv'

df = pd.read_csv(csv_filename, on_bad_lines='skip')

df.columns = df.columns.str.strip().str.replace('"', '')
df_cleaned = df.dropna()
df_cleaned = df_cleaned.drop_duplicates()

# Ajouter l'id de l'user aux playlists 'Starred' et 'Liked from Radio'
def update_playlist_name(row):
    if row['playlistname'] == 'Starred':
        return row['playlistname'] + row['user_id']
    elif row['playlistname'] == 'Liked from Radio':
        return row['playlistname'] + row['user_id']
    return row['playlistname']

# Appliquer la fonction à chaque ligne du DataFrame
df_cleaned['playlistname'] = df_cleaned.apply(update_playlist_name, axis=1)

# la colonne user_id n'est pas utile
# df_cleaned = df_cleaned.drop('user_id', axis=1)

In [3]:
df_cleaned.head()

,user_id,artistname,trackname,playlistname
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [4]:
print(df_cleaned.describe())

                                 user_id artistname trackname  \
count                           12856835   12856835  12856835   
unique                             15914     289603   2004523   
top     4398de6902abde3351347b048fcdc287  Daft Punk     Intro   
freq                              295274      36086      6675   

                                   playlistname  
count                                  12856835  
unique                                   165459  
top     Starred4398de6902abde3351347b048fcdc287  
freq                                      47362  


## Compter le nombre de colonnes

In [5]:
value_counts = df_cleaned['playlistname'].value_counts()

min_frequency = value_counts.min()
max_frequency = value_counts.max()
average = value_counts.mean()
median = value_counts.median()

print("Minimum frequency:", min_frequency)
print("Maximum frequency:", max_frequency)
print("Average frequency:", average)
print("Median frequency:", median)

Minimum frequency: 1
Maximum frequency: 47362
Average frequency: 77.70405357218405
Median frequency: 20.0


## Enlever les playlists qui ont moins de sons que la valeur médiane 

In [6]:
filtered_df = df_cleaned.groupby('playlistname').filter(lambda x: len(x) > median)
print(filtered_df.describe())

                                 user_id artistname trackname  \
count                           11972312   11972312  11972312   
unique                             14694     276343   1900565   
top     4398de6902abde3351347b048fcdc287  Daft Punk     Intro   
freq                              295274      34553      5859   

                                   playlistname  
count                                  11972312  
unique                                    81916  
top     Starred4398de6902abde3351347b048fcdc287  
freq                                      47362  


In [7]:
value_counts = filtered_df['playlistname'].value_counts()

min_frequency = value_counts.min()
max_frequency = value_counts.max()
average = value_counts.mean()
median = value_counts.median()

print("Minimum frequency:", min_frequency)
print("Maximum frequency:", max_frequency)
print("Average frequency:", average)
print("Median frequency:", median)

Minimum frequency: 21
Maximum frequency: 47362
Average frequency: 146.15352312124617
Median frequency: 51.0


## Récupérer les caractéristiques des sons grâce à l'API de Spotify

### credentials

In [8]:
client_id = 'clientid'
client_secret = 'clientsecret'
redirect_uri = 'http://localhost:8888/callback'

spotify_client = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope='user-library-read playlist-modify-public'))

In [9]:
test_df = filtered_df.head(1000) # 1000 sons = 3-4 minutes pour avoir les ids
test_df.head(10)

,user_id,artistname,trackname,playlistname
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010
5,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,All Be Okay,HARD ROCK 2010
6,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Band On The Run,HARD ROCK 2010
7,9cc0cfd4d7d7885102480dd99e7a90d6,Joe Echo,Beautiful,HARD ROCK 2010
8,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,"Blackbird - Live at CitiField, NYC - Digital A...",HARD ROCK 2010
9,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Bright Side,HARD ROCK 2010


In [10]:
def get_track_ids(track_names, artist_names, dataframe):
    track_ids = []
    indices_to_delete = [] 
    for i, (track_name, artist_name) in enumerate(zip(track_names, artist_names)):
        results = spotify_client.search(q=f'track:{track_name} artist:{artist_name}', type='track', limit=1)
        if results['tracks']['items']:
            track_id = results['tracks']['items'][0]['id']
            track_ids.append(track_id) 
        else:
            indices_to_delete.append(i)

    if indices_to_delete: 
        dataframe = dataframe.drop(dataframe.index[indices_to_delete]).reset_index(drop=True)
        
    return track_ids, dataframe

In [11]:
track_ids, test_df = get_track_ids(test_df['trackname'].tolist(), test_df['artistname'].tolist(), test_df)
print(test_df)

                              user_id                        artistname  \
0    9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   
1    9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
2    9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
3    9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   
4    9cc0cfd4d7d7885102480dd99e7a90d6                            Lissie   
..                                ...                               ...   
885  07f0fc3be95dcd878966b1f9572ff670                          Deftones   
886  07f0fc3be95dcd878966b1f9572ff670                        Audioslave   
887  07f0fc3be95dcd878966b1f9572ff670                            John 5   
888  07f0fc3be95dcd878966b1f9572ff670                               C2C   
889  07f0fc3be95dcd878966b1f9572ff670                 Infected Mushroom   

                                             trackname    playlistname  
0                 (The Ang

In [12]:
split = [track_ids[i:i+50] for i in range(0, len(track_ids), 50)]
song_data = []
fieldnames = ['valence', 'year', 'acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
              'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'name', 
              'popularity', 'release_date', 'speechiness', 'tempo', 'uri']

for track_ids in split:
    audio_features = spotify_client.audio_features(track_ids)
    track_details = spotify_client.tracks(track_ids)['tracks']

    for i, item in enumerate(track_ids):
        audio_feature = audio_features[i]
        details = track_details[i]

        artists = ', '.join([artist['name'] for artist in details['artists']])

        song_info = {
            'artists': artists,
            'key': audio_feature['key'],
            'danceability': audio_feature['danceability'],
            'liveness': audio_feature['liveness'],
            'valence': audio_feature['valence'],
            'year': details['album']['release_date'][:4],
            'acousticness': audio_feature['acousticness'],
            'duration_ms': audio_feature['duration_ms'],
            'energy': audio_feature['energy'],
            'explicit': int(details['explicit']),
            'instrumentalness': audio_feature['instrumentalness'],
            'loudness': audio_feature['loudness'],
            'mode': audio_feature['mode'],
            'popularity': details['popularity'],
            'speechiness': audio_feature['speechiness'],
            'tempo': audio_feature['tempo'],
            'release_date': details['album']['release_date']
        }

        song_data.append(song_info)

for i, data in enumerate(song_data):
    for col, value in data.items():
        if col not in test_df.columns:
            test_df[col] = None
        test_df.at[i, col] = value


In [13]:
test_df.head()
print(len(test_df))

890


# Ajout de la colonne l'URI de la track dans copy_test_df

In [ ]:
test_df.head(1)

# copie du dataframe
copy_test_df = test_df.copy()

print("colonnes avant : \n", copy_test_df.columns)

# Ajout une colonne URI
def get_track_uri(track_name, artist_name, spotify_client):
    results = spotify_client.search(q=f'track:{track_name} artist:{artist_name}', type='track', limit=1)
    if results['tracks']['items']:
        track_uri = results['tracks']['items'][0]['uri']
        return track_uri
    else:
        return None

# Ajouter une colonne 'uri' contenant les identifiants URI dans copy_test_df
copy_test_df['uri'] = copy_test_df.apply(lambda row: get_track_uri(row['trackname'], row['artistname'], spotify_client), axis=1)

print("colonnes apères : \n ", copy_test_df.columns)


In [32]:
print(copy_test_df.columns)
copy_test_df.head(1)

Index(['user_id', 'artistname', 'trackname', 'playlistname', 'artists', 'key',
       'danceability', 'liveness', 'valence', 'year', 'acousticness',
       'duration_ms', 'energy', 'explicit', 'instrumentalness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'release_date', 'uri'],
      dtype='object')


,user_id,artistname,trackname,playlistname,artists,key,danceability,liveness,valence,year,...,energy,explicit,instrumentalness,loudness,mode,popularity,speechiness,tempo,release_date,uri
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010,Elvis Costello,4,0.577,0.226,0.901,1977,...,0.636,0,0.00016,-9.825,1,40,0.0491,134.768,1977-07-22,spotify:track:0GmLrYUBXDC5vti77zBZfJ


# Ajout de la colonne l'URI de la track dans spotify_data

ça fonctionne ?

In [44]:
spotify_data = pd.read_csv('./data/data.csv.zip')

# Data format
spotify_data['artists'] = spotify_data['artists'].str.replace("['", "", regex=False)
spotify_data['artists'] = spotify_data['artists'].str.replace("']", "", regex=False)
spotify_data['artists'] = spotify_data['artists'].str.replace("'", "", regex=False)

spotify_data = spotify_data.rename(columns=lambda x: x.strip().replace('"', ''))
spotify_data = spotify_data.rename(columns={'name': 'trackname', 'artists': 'artistname'})

print("taille : ", len(spotify_data))
print("colonnes avant : \n", spotify_data.columns)

taille :  170653
colonnes avant : 
 Index(['valence', 'year', 'acousticness', 'artistname', 'danceability',
       'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'trackname', 'popularity',
       'release_date', 'speechiness', 'tempo'],
      dtype='object')


In [46]:
# copie du dataframe mais que les 5 premiers sons
copy_spotify_data = spotify_data.head(10)
print("taille de spotify_data :", len(copy_spotify_data))
print("colonnes avant : \n", copy_spotify_data.columns)

# Ajouter une colonne 'uri' contenant les identifiants URI dans copy_spotify_data
copy_spotify_data['uri'] = copy_spotify_data.apply(lambda row: get_track_uri(row['trackname'], row['artistname'], spotify_client), axis=1)

print("colonnes après : \n", copy_spotify_data.columns)

taille de spotify_data : 10
colonnes avant : 
 Index(['valence', 'year', 'acousticness', 'artistname', 'danceability',
       'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'trackname', 'popularity',
       'release_date', 'speechiness', 'tempo'],
      dtype='object')
colonnes après : 
 Index(['valence', 'year', 'acousticness', 'artistname', 'danceability',
       'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'trackname', 'popularity',
       'release_date', 'speechiness', 'tempo', 'uri'],
      dtype='object')


C:\Users\Liant\AppData\Local\Temp\ipykernel_10044\959946254.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_spotify_data['uri'] = copy_spotify_data.apply(lambda row: get_track_uri(row['trackname'], row['artistname'], spotify_client), axis=1)


In [48]:
copy_spotify_data.head(1)

,valence,year,acousticness,artistname,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,trackname,popularity,release_date,speechiness,tempo,uri
0,0.0594,1921,0.982,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954,spotify:track:6ZVPpYGLxrQySjXkCxtziR


# CHERCHER UN SON EN COMMUN DANS DATA DES PLAYLIST ET L'AUTRE

à faire

## Diviser le dataset en un set de test et un set d'entrainement

In [15]:
# enregister direct dans un csv